using the Hugging Face Transformers library to train a BERT model for summarizing paragraphs

Step 1: Collect dataset

In [1]:
import pandas as pd

# Load dataset
df = pd.read_csv('train.csv')

Step 2: Preprocess dataset

In [5]:
from transformers import AutoTokenizer

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode text and summary
encoded_data = tokenizer.batch_encode_plus(df['Article'], 
                                           df['Summary'], 
                                           padding=True, 
                                           truncation=True, 
                                           max_length=512, 
                                           return_tensors='pt')

# Convert data to PyTorch tensors
input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']
labels = encoded_data['labels']

TypeError: batch_text_or_text_pairs has to be a list or a tuple (got <class 'pandas.core.series.Series'>)

Step 3: Split dataset into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(input_ids, labels, test_size=0.2, random_state=42)
train_masks, test_masks, _, _ = train_test_split(attention_masks, input_ids, test_size=0.2, random_state=42)

Step 4: Train a Machine Learning model

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AdamW

# Initialize BERT model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

# Set up optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 2
total_steps = len(X_train) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Train BERT model
for epoch in range(epochs):
    model.train()
    for i in range(len(X_train)):
        input_id = X_train[i].to(device)
        attention_mask = train_masks[i].to(device)
        label = y_train[i].to(device)
        model.zero_grad()
        output = model(input_id, attention_mask=attention_mask, labels=label)
        loss = output.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

Step 5: Evaluate model performance

In [ ]:
from sklearn.metrics import mean_squared_error

# Evaluate model performance on testing set
model.eval()
with torch.no_grad():
    y_pred = []
    for i in range(len(X_test)):
        input_id = X_test[i].to(device)
        attention_mask = test_masks[i].to(device)
        output = model(input_id, attention_mask=attention_mask)
        y_pred.append(output.logits.item())
mse = mean_squared_error(y_test, y_pred)
print(f'Mean squared error: {mse}')

Step 6: Deploy model for use in summarizing new paragraphs

In [ ]:
# Convert new paragraph to PyTorch tensor
new_text = 'This is a new paragraph to summarize.'
encoded_text = tokenizer.encode_plus(new_text, padding=True, truncation=True, max_length=512, return_tensors='pt')
input_id = encoded_text['input_ids'].to(device)
attention_mask = encoded_text['attention_mask'].to(device)

# Use trained model to generate summary
model.eval()
with torch.no_grad():
    output = model(input_id, attention_mask=attention_mask)
new_summary = output.logits.item()
print(f'Summary: {new_summary}')